In [116]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from numpy.lib.function_base import kaiser
from PIL import Image
import pickle

In [140]:
%run OpenSYK_functions.ipynb

In [118]:
## import data ##
DATA=[]
i,j= 0,0
m= -1
for a in range(len_k):
  for b in range(len_l):

    Model = OpenSYK(P["w"][0], P["gamma/w"][0], P["w/g"][a],\
    P["gamma/beta"][b], P["N"][m],  1 )

    N= Model.N
    name = 'DataBatchParallel/N_%s/gain/w%.3f_w_g%.3f_'%(Model.N, Model.w, Model.w_g)+'N%.1f_'%(N)\
              +'gamma_w%.3f_gamma_beta%.6f'%(Model.gamma_w, Model.gamma_beta)
    name_lg = 'DataBatchParallel/N_%s/loss_gain/w%.3f_g%.3f_'%(Model.N, Model.w, Model.g)+\
        'N%.1f_'%(N)+'gamma%.3f_beta%.6f'\
         %(Model.gamma, Model.beta)
    with open(name_lg+'.pickle','rb') as handle:
      DATA.append(pickle.load( handle))

DATA = np.array(DATA)
DATA = DATA.reshape(len_k,len_l)
DATA.shape


(9, 9)

In [43]:
size =24

In [148]:
#Purity sampling 

samples = 350
for x in range(9):
  for y in range(9):  
    Model = create_Model(0,0,x,y,-1)
    N= Model.N
    fig = plt.figure(figsize=(10,10)) # Figureを作成
    ax1 = fig.add_subplot(1,1,1) # Axesを作成
    pu=np.array(DATA[x][y]["purity"])
    av= np.mean(pu,axis=0),np.var(pu, axis=0)
    Pty= [DATA[x][y]["purity_infty"]]*len(times)
    for n in range(samples):
        ax1.plot(times,pu[n],linewidth=0.12, c='grey')
    ax1.errorbar(times, av[0], av[1], linestyle='-.',label= r'$<P>$')
    ax1.plot(times,av[0], c='y',linewidth= 2)
    ax1.plot(times,Pty,c='y', label=r'$P_\infty$')
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    
    plt.ylabel("Purity of $\\rho(t)$",fontsize=size)
    plt.xlabel("time", fontsize=size)
    
    name_m = "ImagesBatchParallel/Purity/loss_gain/N_%s"%(N)+"/Purity "\
    'w%.3f_g%.5f_'%(Model.w, Model.g)+\
        'N%.1f_'%(N)+'gamma%.3f_beta%.6f.jpg'\
         %(Model.gamma, Model.beta)
    title = "Purity "+ 'w%.3f_g%.5f_'%(Model.w, Model.g)+\
        'N%.1f_'%(N)+'gamma%.3f_beta%.6f'\
         %(Model.gamma, Model.beta)
    ax1.tick_params(labelsize=20)
    plt.title(title, fontsize= size)
    plt.legend(fontsize=size)
    plt.savefig(name_m)
    plt.close(fig)
    

In [151]:
#Entropy sampling 
i,j= 0,0

for k in range(len_k):
  for l in range(len_l):
    Model = create_Model(i,j,k,l,m)
    N= Model.N
    entropy_max= np.log(2**(N//2+1))
    
    fig = plt.figure(figsize=(10,10)) # Figureを作成
    ax1 = fig.add_subplot(1,1,1) # Axesを作成
    
    ent=np.array(DATA[k][l]["entropy"])
    av= np.mean(ent,axis=0),np.var(ent, axis=0)
    eig =np.array(DATA[k][l]["eigenvalues"])
    av_eig=  np.mean(eig, axis=0)  
    S= [S_infty(Model.beta,av_eig)/entropy_max]*len(times)

    
    for n in range(samples):
        ax1.plot(times,ent[n],linewidth=0.12, c='grey')
    ax1.errorbar(times, av[0], av[1], linestyle='-.',label= r'$<S>$')
    ax1.plot(times,av[0], c='y',linewidth= 2)
    ax1.plot(times, S, c='orange', linewidth= 2, label=r'$ S_\infty $')
   
    plt.ylabel(r"$S(\rho(t))$",fontsize=size)
    plt.xlabel("time", fontsize=size)
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    
    name_lg = "ImagesBatchParallel/Entropy/N_%s/loss_gain"%(N)+"/Entropy "\
    'w%.3f_g%.5f_'%(Model.w, Model.g)+\
        'N%.1f_'%(N)+'gamma%.3f_beta%.6f.jpg'\
         %(Model.gamma, Model.beta)
    title = "Entropy "+ 'w%.3f_g%.5f_'%(Model.w, Model.g)+\
        'N%.1f_'%(N)+'gamma%.3f_beta%.6f'\
         %(Model.gamma, Model.beta)
    
    plt.title(title, fontsize= size)
    plt.legend(fontsize = size)
    ax1.tick_params(labelsize=20)
    
    plt.savefig(name_lg)
    plt.close(fig)

In [ ]:
#Occupation number sampling 
i,j= 0,0
N = P["N"][m]
for k in range(len_k):
  for l in range(len_l):
    fig = plt.figure(figsize=(10,10))
    Model = create_Model(i,j,k,l,m)
    av_occ=  (np.array(DATA[k][l]["counts_d"]).sum(axis=0))/50 
    d_min = [min(av_occ[0])]*len(times)
    
    for n in range(P["AverageNumber"]):
      plt.plot(times,DATA[k][l]["counts_d"][n][0],linewidth=0.2)
    plt.plot(times, av_occ[0],label="average occupation number prob.")
    plt.plot(times, d_min, label="min. occupation prob. %.2f"%(d_min[0]))

    title= "Occupation number prob."+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
              +'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f'%(Model.gamma_w, Model.gamma_beta)
    name_g= "ImagesBatchParallel/OccupationNumber/N_%s"%(N)\
    +"/gain/OccupationN "+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
                +'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f.jpg'%(Model.gamma_w, Model.gamma_beta)
    name_lg= "ImagesBatchParallel/OccupationNumber/N_%s"%(N)\
    +"/loss_gain/OccupationN "+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
                +'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f.jpg'%(Model.gamma_w, Model.gamma_beta)
    plt.ylabel("  $d^\dagger d$",fontsize=18)
    plt.xlabel("time", fontsize=18)
    plt.title(title, fontsize= size)
    plt.legend(fontsize = size)
    plt.savefig(name_lg)
    plt.close(fig)

In [ ]:
#L eigenvalues

i,j= 0,0

N = P["N"][m]
for k in range(len_k):
  for l in range(len_l):
    fig = plt.figure(figsize=(10,10))
    Model = create_Model(i,j,k,l,m)
    w = Model.w
    eig =np.array(DATA[k][l]["eigenvalues_L"])/w
    Re_eig= np.real(eig)
    Im_eig= np.imag(eig)
    
    plt.scatter(Re_eig, Im_eig, c ="b",s=10)
    
    plt.ylabel("Im"+"$L$",fontsize=size)
    plt.xlabel("Re"+"$L$", fontsize=size)
    
    title = "Eigenvalue distribution of L "+\
    'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f'%(Model.gamma_w, Model.gamma_beta)
    name_g= "ImagesBatchParallel/Eigenvalues_L/N_%s"%(N)+"/gain/Eigenvalues_L "\
    +'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f.jpg'%(Model.gamma_w, Model.gamma_beta)
    name_lg= "ImagesBatchParallel/Eigenvalues_L/N_%s"%(N)+"/loss_gain/Eigenvalues_L "\
    +'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f.jpg'%(Model.gamma_w, Model.gamma_beta)
    plt.title( title, fontsize = size )
    plt.savefig(name_lg)
    plt.close(fig)



In [ ]:
# Particle number
m= 0
# Multiple animation 
N = P["N"][m]

for k in range(len_k):

  for l in range(len_l):

    Model = create_Model(i,j,k,l,m)

    fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, figsize=(14,12))

    # Purity
    purity_min = [1/2**(N//2+1)]*len(times)
    av_purity=  (np.array(DATA[k][l]["purity"]).sum(axis=0))/(50*purity_min[0]) 
    for n in range(P["AverageNumber"]):
      ax1.plot(times,(DATA[k][l]["purity"][n])/purity_min[0],linewidth=0.2)
    ax1.plot(times, av_purity,label="average purity")
    #ax1.plot(times,purity_min, label="min. purity")

    ax1.set_xlabel('time', fontsize=18)
    ax1.set_ylabel(r'$P(t)/P_{min}$', fontsize=15)
    ax1.set_title('Purity',fontsize=21)
    ax1.legend(fontsize= size)

    #Von Neumann entropy
    entropy_max= [np.log(2**(N//2+1))]*len(times)
    av_entropy=  (np.array(DATA[k][l]["entropy"]).sum(axis=0))/(50*entropy_max[0]) 
    
    for q in range(P["AverageNumber"]):
      ax2.plot(times,(DATA[k][l]["entropy"][q])/entropy_max[0],linewidth=0.2)
    ax2.plot(times, av_entropy,label="average entropy")
    #ax2.plot(times, entropy_max, label="max. entropy")

    ax2.set_ylabel("$S(t)/S_{max}$",fontsize=18)
    ax2.set_xlabel("time", fontsize=18)
    ax2.set_title("Von Neumann entropy ",fontsize=21)
    ax2.legend(fontsize= size)
   

    #Entropy of the system
    av_occ=  (np.array(DATA[k][l]["counts_d"]).sum(axis=0))/50 
    d_min = [min(av_occ[0])]*len(times)
    
    for r in range(P["AverageNumber"]):
      ax3.plot(times,DATA[k][l]["counts_d"][r][0],linewidth=0.2)
    ax3.plot(times, av_occ[0],label="average number")
    ax3.plot(times, d_min, label="min. %.2f"%(d_min[0]))

    ax3.set_ylabel("  $d^\dagger d$",fontsize=18)
    ax3.set_xlabel("time", fontsize=18)
    ax3.set_title("Occupation number prob.",fontsize=21)
    ax3.legend(fontsize = size)
    

    #Eigen values of the Liouvillian superoperator
    w = Model.w
    eig =np.array(DATA[k][l]["eigenvalues_L"])/w
    Re_eig= np.real(eig)
    Im_eig= np.imag(eig)
    
    ax4.scatter(Re_eig, Im_eig, c ="b",s=10)
    
    

    ax4.set_ylabel("Im"+r"$\frac{L}{\omega}$" ,fontsize=18)
    ax4.set_xlabel("Re"+r"$\frac{L}{\omega}$" , fontsize=18)
    ax4.set_title("Eigenvalue distribution of L ", fontsize=21)
  
    title = 'Quantities '+r'$\omega=%.3f$, '%(Model.w)+r'$\frac{\omega}{g}=$'+'%.3f, '%(Model.w_g)\
                +'N=%.1f, '%(N)+r'$\frac{J}{\omega}=$'+'%.3f, '%(Model.gamma_w)+ \
     r'$\frac{J}{\beta}=$'+'%.6f'%(Model.gamma_beta)
    
    name_g = 'ImagesBatchParallel/Quantities/N_%s/'%(N)\
                +'gain/Quantities w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
                +'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f.jpg'%(Model.gamma_w, Model.gamma_beta)
    
    name_lg = 'ImagesBatchParallel/Quantities/N_%s/'%(N)\
                +'loss_gain/Quantities w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
                +'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f.jpg'%(Model.gamma_w, Model.gamma_beta)
    fig.suptitle(title, fontsize=23)
    plt.savefig(name_lg)
    plt.close(fig)
    
    
    



In [ ]:
#deviation from analytical result (entropy)
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import mpl_toolkits.mplot3d

fig, ax = plt.subplots(subplot_kw={"projection": "3d"},figsize=(16,16))

K= np.arange(len_k)
L = np.arange(len_l)
X=np.log(P['w/g'])
Y= np.log(P['gamma/beta'])
X= np.round(X, decimals=1)
Y= np.round(Y, decimals=1)
Z = np.array([[np.array(DATA[k][l]['error_s']).sum(axis=0)/50 for k in K ]for l in L])

Z=Z.sum(axis=2)/50
Where_Nan = np.isnan(Z)
Z[Where_Nan]= 1

for d in range(len_k):
    ax.plot(X,np.ones(len_k)*Y[d],np.log(Z[:][d]) )
    ax.scatter(X,np.ones(len_k)*Y[d],np.log(Z[:][d]) )

    #ax.set_zscale('log')

ax.set_zlabel(r'$\log|S_{num}(\infty)-S_{an}(\infty)|$',fontsize=20, labelpad= 25.2 )
ax.set_xlabel('log(w/g)',fontsize=20, labelpad= 25.2 )
ax.set_ylabel(r'$\log(J/\beta)$', fontsize=20, labelpad= 25.2 )
ax.view_init(azim=25, elev=40)

ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)
ax.tick_params(axis='z', labelsize=20)
ax.set_xticklabels(X, fontsize=20)
ax.dist = 11
ax.set_title("Simulation error for"+r" $S(\infty)$,"+" N=%.f"%(Model.N), fontsize=29)
#ax.set_yticklabels(Y, fontsize=20)
name = 'ImagesBatchParallel/S_infty.jpg'
plt.savefig(name)

In [ ]:
#deviation from analytical result (purity)
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import mpl_toolkits.mplot3d
N= P['N'][m]
fig, ax = plt.subplots(subplot_kw={"projection": "3d"},figsize=(16,16))

K= np.arange(len_k)
L = np.arange(len_l)
X=np.log(P['w/g'])
Y= np.log(P['gamma/beta'])
X= np.round(X, decimals=1)
Y= np.round(Y, decimals=1)
Z = np.array([[np.array(DATA[k][l]['error_p']).sum(axis=0)/50 for k in K ]for l in L])

Z=Z.sum(axis=2)/50
Where_Nan = np.isnan(Z)
Z[Where_Nan]= 1

for d in range(len_k):
    ax.plot(X,np.ones(len_k)*Y[d],np.log(Z[:][d]) )
    ax.scatter(X,np.ones(len_k)*Y[d],np.log(Z[:][d]) )


ax.set_zlabel(r'$\log|P_{num}(\infty)-P_{an}(\infty)|$',fontsize=20, labelpad= 25.2 )
ax.set_xlabel('log(w/g)',fontsize=20, labelpad= 25.2 )
ax.set_ylabel(r'$\log(J/\beta)$', fontsize=20, labelpad= 25.2 )
ax.view_init(azim=25, elev=40)

ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)
ax.tick_params(axis='z', labelsize=20)
ax.set_xticklabels(X, fontsize=20)
ax.dist = 11

ax.set_title("Simulation error for"+r" $P(\infty)$,"+" N=%.f"%(N), fontsize=29)
#ax.set_yticklabels(Y, fontsize=20)
name = 'ImagesBatchParallel/P_infty.jpg'
plt.savefig(name)

In [ ]:
# Purity 

i,j,m =0,0,1
N = P["N"][m]
Images= []
for k in range(len_k):
  for l in range(len_l):
    Model = create_Model(i,j,k,l,m)
    name_m= "ImagesBatchParallel/Purity/N_%s"%(N)+\
    "/Purity "+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f.jpg'%(Model.gamma_w, Model.gamma_beta)
    Images.append(Image.open(name_m))


Images = np.array(Images)
Images = Images.reshape(len_k, len_l)
Imagesbeta= Images.T

#Purity

for k in range(len_k):
    
    Model = create_Model(i,j,k,l,m)
    
    name_w= 'Animations/Ani_Purity/N_%s'%(N)+\
    '/fixed_w/AniPurity '+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f.gif'%(Model.gamma_w)
    name_g = 'Animations/Ani_Purity/N_%s'%(N)+\
    '/fixed_g/AniPurity '+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f.gif'%(Model.gamma_w)
    
    Images[k][0].save(name_w,\
                      save_all=True, append_images= Images[k][1:],  optimize=False, duration= 1090, loop=0)
    Imagesbeta[k][0].save(name_g,\
                      save_all=True, append_images= Imagesbeta[k][1:],  optimize=False, duration= 1090, loop=0)
  

In [ ]:
#Entropy

i,j,m= 0,0,1
Images= []
N = P["N"][m]
for k in range(len_k):
  for l in range(len_l):
    Model = create_Model(i,j,k,l,m)
    name= "ImagesBatchParallel/Entropy/N_%s"%(N) \
    +"/Entropy "+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f.jpg'%(Model.gamma_w, Model.gamma_beta)
    Images.append(Image.open(name))


Images = np.array(Images)
Images = Images.reshape(len_k, len_l)
Imagesbeta= Images.T
#Entropy


for k in range(len_k):
    
    Model = create_Model(i,j,k,l,m)
    
    name_w = 'Animations/Ani_Entropy/N_%s'%(N)+\
    '/fixed_w/AniEntropy '+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f.gif'%(Model.gamma_w)
    name_g = 'Animations/Ani_Entropy/N_%s'%(N)+\
    '/fixed_g/AniEntropy '+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f.gif'%(Model.gamma_w)
    
    Images[k][0].save(name_w,\
                      save_all=True, append_images= Images[k][1:],  optimize=False, duration= 1090, loop=0)
    Imagesbeta[k][0].save(name_g,\
                      save_all=True, append_images= Imagesbeta[k][1:],  optimize=False, duration= 1090, loop=0)
  

In [ ]:
#Occupation number

i,j= 0,0
Images= []

for k in range(len_k):
  for l in range(len_l):
    Model = create_Model(i,j,k,l,m)
    name= "ImagesBatchParallel/OccupationNumber/N_%s"%(N)+\
    "/OccupationN "+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f.jpg'%(Model.gamma_w, Model.gamma_beta)
    Images.append(Image.open(name))


Images = np.array(Images)
Images = Images.reshape(len_k, len_l)
Imagesbeta= Images.T

#Occupation Number


for k in range(len_k):
    
    Model = create_Model(i,j,k,l,m)
    
    name_w = 'Animations/Ani_OccupationNumber/N_%s'%(N)+\
    '/fixed_w/AniOccupationN '+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f.gif'%(Model.gamma_w)
    name_g = 'Animations/Ani_OccupationNumber/N_%s'%(N)+\
    '/fixed_g/AniOccupationN '+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f.gif'%(Model.gamma_w)
    
    Images[k][0].save(name_w,\
                      save_all=True, append_images= Images[k][1:],  optimize=False, duration= 1090, loop=0)

    Imagesbeta[k][0].save(name_g,\
                      save_all=True, append_images= Imagesbeta[k][1:],  optimize=False, duration= 1090, loop=0)
  

In [ ]:
# Lindbladian eigenvalues

i,j= 0,0
Images= []
for k in range(len_k):

  for l in range(len_l):
    Model = create_Model(i,j,k,l,m)
    name= "ImagesBatchParallel/Eigenvalues_L/N_%s"%(N)+\
    "/Eigenvalues_L "+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f.jpg'%(Model.gamma_w, Model.gamma_beta)
    Images.append(Image.open(name))


Images = np.array(Images)
Images = Images.reshape(len_k, len_l)
Imagesbeta= Images.T




for k in range(len_k):
    
    Model = create_Model(i,j,k,l,m)
    
    name_w = 'Animations/Ani_Eigenvalues_L/N_%s'%(N)+\
    '/fixed_w/AniEL '+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f.gif'%(Model.gamma_w)
    name_g = 'Animations/Ani_Eigenvalues_L/N_%s'%(N)+\
    '/fixed_g/AniEL '+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f.gif'%(Model.gamma_w)
    
    Images[k][0].save(name_w,\
                      save_all=True, append_images= Images[k][1:],  optimize=False, duration= 1090, loop=0)
    Imagesbeta[k][0].save(name_g,\
                      save_all=True, append_images= Imagesbeta[k][1:],  optimize=False, duration= 1090, loop=0)
  


In [ ]:
#multiple graphs

i,j= 0,0
Images= []
for k in range(len_k):

  for l in range(len_l):
    Model = create_Model(i,j,k,l,m)
    name_m = 'ImagesBatchParallel/Quantities/N_%s/loss_gain/'%(N)\
    +'Quantities w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f_gamma_beta%.6f.jpg'%(Model.gamma_w, Model.gamma_beta)
    Images.append(Image.open(name_m))


Images = np.array(Images)
Images = Images.reshape(len_k, len_l)
Imagesbeta = Images.T




for k in range(len_k):
    Model = create_Model(i,j,k,l,m)
    Model_g = create_Model(i,j,k,k,m)
    name_w_LG = 'Animations/Ani_Quantities/N_%s'%(N)+\
    '/loss_gain/fixed_w/AniQ '+'w%.3f_w_g%.3f_'%(Model.w, Model.w_g)\
    +'N%.1f_'%(N)+'gamma_w%.3f.gif'%(Model.gamma_w)
    
    name_g_LG = 'Animations/Ani_Quantities/N_%s'%(N)+\
    '/loss_gain/fixed_g/AniQ '+'w%.3f_'%(Model_g.w)\
    +'N%.1f_'%(N)+'beta%.5f.gif'%(Model_g.beta)
    
    Images[k][0].save(name_w_LG,\
                      save_all=True, append_images= Images[k][1:],  optimize=False, duration= 1090, loop=0)
    Imagesbeta[k][0].save(name_g_LG,\
                      save_all=True, append_images= Imagesbeta[k][1:],  optimize=False, duration= 1090, loop=0)
  
 
    